In [1]:
#*******************************************************************************************
 #
 #  File Name:  NoSQLAnalysis.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, NoSQLAnalysis.ipynb, uses the Python module,
 #      PyMongo to analyze restaurant ratings data in a MongoDB database, uk_food, 
 #      for the editors of a food magazine, Eat Safe, Love.
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  09/18/2023      Initial Development                     N. James George
 #
 #******************************************************************************************/

import PyFunctions as function
import PyLogFunctions as log_function
import PyLogSubRoutines as log_subroutine

import pandas as pd

from pprint import pprint
from pymongo import MongoClient

In [2]:
CONSTANT_LOCAL_FILE_NAME \
    = 'NoSQLAnalysis.ipynb'


log_subroutine \
    .SetLogMode \
        (False)

log_subroutine \
    .SetDebugMode \
        (False)

log_subroutine \
    .SetImageMode \
        (False)


log_subroutine \
    .BeginProgramExecution \
        ('NoSQLAnalysis')

# <br> **Section 1: Jupyter Notebook Set Up**

## **MongoDB Client**

In [3]:
# This line of code creates an instance of a MongoDB client and
# assigns it to a variable.
currentPyMongoClientObject \
    = MongoClient \
        (port \
             = 27017)


log_function \
    .DebugReturnObjectWriteObject \
        (currentPyMongoClientObject)

## **MongoDB Database Confirmation**

In [4]:
# These lines of code list all the databases in MongoDB and confirm 
# the existence of the database, uk_food.
log_subroutine \
    .PrintAndLogWriteText \
        ('Here is a List of the current MongoDB Databases:\n'
          + str \
                (currentPyMongoClientObject \
                    .list_database_names()))


if 'uk_food' in currentPyMongoClientObject.list_database_names():
     
    log_subroutine \
        .PrintAndLogWriteText \
            ("\nThe MongoDB Database, 'uk_food', is in the List " \
             + 'of MongoDB Databases.')
    
else:
    
    log_subroutine \
        .PrintAndLogWriteText \
            ("\nThe MongoDB Database, 'uk_food', is NOT in the List. " \
             + 'Please run NoSQLSetup.ipynb before proceeding!') 

Here is a List of the current MongoDB Databases:
['admin', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']

The MongoDB Database, 'uk_food', is in the List of MongoDB Databases.


## **MongoDB Database**

In [5]:
# This line of code creates a PyMongo Database Object for the database, 
# uk_food, and assigns it to a variable.
currentPyMongoDatabaseObject \
    = currentPyMongoClientObject \
        .uk_food
    
    
log_function \
    .DebugReturnObjectWriteObject \
        (str(currentPyMongoDatabaseObject))

## **MongoDB Collection Confirmation**

In [6]:
# These lines of code list all the Collections in the MongoDB database, 
# uk_food, and confirm the existence of the Collection, establishments.
log_subroutine \
    .PrintAndLogWriteText \
        ('Here is a List of the current Collections ' \
         + 'in the MongoDB Database, uk_food:\n' \
         + str \
            (currentPyMongoDatabaseObject \
                .list_collection_names()))


if 'establishments' in currentPyMongoDatabaseObject.list_collection_names():
     
    log_subroutine \
        .PrintAndLogWriteText \
            ("\nThe Collection, 'establishments', " \
             + "is in the MongoDB Database, 'uk_food'.")
    
else:
    
    log_subroutine \
        .PrintAndLogWriteText \
            ("\nThe Collection, 'establishments', " \
             + "is NOT in the MongoDB database, 'uk_food'. " \
             + 'Please run NoSQLSetup.ipynb before proceeding!') 

Here is a List of the current Collections in the MongoDB Database, uk_food:
['establishments']

The Collection, 'establishments', is in the MongoDB Database, 'uk_food'.


## **MongoDB Collection**

In [7]:
# This line of code creates a PyMongo Collection Object for the collection,
# establishments, and assigns it to a variable.
currentPyMongoCollectionObject \
    = currentPyMongoDatabaseObject \
        .establishments


log_function \
    .DebugReturnObjectWriteObject \
        (str(currentPyMongoCollectionObject))

# <br> **Section 2: Exploratory Analysis**

## **1. Which establishments have a hygiene score equal to 20?**

### Number of Establishments from `count_documents`

In [8]:
# These lines of code query and display the number of establishments 
# with the lowest hygiene score, 20.
queryDictionary \
    = {'scores.Hygiene': 20}

numberOfEstablishmentsInteger \
    = currentPyMongoCollectionObject \
        .count_documents \
            (queryDictionary)

log_subroutine \
    .PrintAndLogWriteText \
        ('There are {:,} establishments with a hygiene score equal to 20.' \
            .format \
                (numberOfEstablishmentsInteger))

There are 41 establishments with a hygiene score equal to 20.


### Display First Document

In [9]:
# This query returns all the documents where the restaurant 
# has a hygiene score equal to 20.
queryResultsDictionaryList \
    = currentPyMongoCollectionObject \
        .find \
            (queryDictionary)


# This line of code displays the first document from the query 
# results.
pprint \
    (queryResultsDictionaryList[0])

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650a6b8c5bc6d3e250bfac6f'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractD

### Worst Hygiene Establishments DataFrame

In [10]:
worstHygieneDataFrame \
    = pd \
        .DataFrame \
            (list \
                (queryResultsDictionaryList))


log_function \
    .DebugReturnObjectWriteObject \
        (worstHygieneDataFrame)

### Number of Rows in Worst Hygiene Establishments DataFrame

In [11]:
log_subroutine \
    .PrintAndLogWriteText \
        ('The number of rows in the Lowest Hygiene Establishments ' \
         + 'DataFrame is {:,}.' \
                .format \
                    (len \
                         (worstHygieneDataFrame)))

The number of rows in the Lowest Hygiene Establishments DataFrame is 41.


### Display Worst Hygiene Establishments DataFrame

In [12]:
function \
    .ReturnStylerObjectStandardFormat \
        (worstHygieneDataFrame.head(10),
         'Table 2.1: Worst Hygiene Establishments',
         hideFlagBooleanParameter = True) \
            .hide \
                ('meta', axis = 1)

_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,links
650a6b8c5bc6d3e250bfac6f,"110,681",0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,,BN21 1BU,,nan,fhrs_0_en-gb,2021-09-23T00:00:00,102,Eastbourne,http://www.eastbourne.gov.uk/foodratings,Customerfirst@eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': 0.27694, 'latitude': 50.769705}",,"4,613.89",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/110681'}]"
650a6b8c5bc6d3e250bfaff0,"612,039",0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,,CO14 8HN,,nan,fhrs_0_en-gb,2022-06-22T00:00:00,120,Tendring,http://www.tendringdc.gov.uk/,fhsadmin@tendringdc.gov.uk,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceInManagement': 30}",FHRS,"{'longitude': 1.278721, 'latitude': 51.857536}",,"4,617.97",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/612039'}]"
650a6b8c5bc6d3e250bfb2fa,"730,933",0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,"7,842",53 Marine Parade East,Clacton On Sea,Essex,,CO15 6AD,,nan,fhrs_0_en-gb,2021-08-13T00:00:00,120,Tendring,http://www.tendringdc.gov.uk/,fhsadmin@tendringdc.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': 1.15927, 'latitude': 51.789429}",,"4,619.66",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/730933'}]"
650a6b8c5bc6d3e250bfb4e8,"172,735",0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,"7,844",4 High Street,Seaford,East Sussex,,BN25 1PG,,1.00,fhrs_1_en-gb,2021-12-23T00:00:00,105,Lewes,http://www.lewes-eastbourne.gov.uk/,ehealth.ldc@lewes-eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': 0.10202, 'latitude': 50.770885}",,"4,620.42",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/172735'}]"
650a6b8c5bc6d3e250bfb4f7,"172,953",0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,,BN25 1HP,,1.00,fhrs_1_en-gb,2021-10-07T00:00:00,105,Lewes,http://www.lewes-eastbourne.gov.uk/,ehealth.ldc@lewes-eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': 0.101446, 'latitude': 50.770724}",,"4,620.44",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/172953'}]"
650a6b8c5bc6d3e250bfbe96,"512,854",0,12/00816/BUTH,Ashby's Butchers,Retailers - other,"4,613",777 Southchurch Road,Southend-On-Sea,Essex,,SS1 2PP,,nan,fhrs_0_en-gb,2022-07-21T00:00:00,893,Southend-On-Sea,http://www.southend.gov.uk,EnvironmentalHealth@southend.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': 0.736349, 'latitude': 51.541448}",,"4,625.57",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/512854'}]"
650a6b8c5bc6d3e250bfc0b7,"1,537,089",0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,,SS1 1BW,,nan,fhrs_0_en-gb,2022-08-03T00:00:00,893,Southend-On-Sea,http://www.southend.gov.uk,EnvironmentalHealth@southend.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20}",FHRS,"{'longitude': 0.7121671, 'latitude': 51.5350065}",,"4,626.20",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/1537089'}]"
650a6b8c5bc6d3e250bfd5e2,"155,648",0,EH/00006058,Golden Palace,Takeaway/sandwich shop,"7,844",7 London Road,Rayleigh,Essex,,SS6 9HN,,nan,fhrs_0_en-gb,2022-03-23T00:00:00,118,Rochford,http://www.rochford.gov.uk,customerservices@rochford.gov.uk,"{'Hygiene': 20, 'Structural

## 2. Which establishments in London have a rating value greater than or equal to 4?

### Number of Establishments from `count_documents`

In [13]:
# These lines of code query and display the number of establishments 
# in London where the 'RatingValue' field has a value greater than
# or equal to 4.
queryDictionary \
    = {'LocalAuthorityName': 
           {'$regex': 'London'}, 
            'RatingValue': {'$gte':4}}

numberOfEstablishmentsInteger \
    = currentPyMongoCollectionObject \
        .count_documents \
            (queryDictionary)

log_subroutine \
    .PrintAndLogWriteText \
        ('There are {:,} establishments with a rating value greater than ' \
             .format \
                (numberOfEstablishmentsInteger) \
         + 'or equal to 4.')

There are 33 establishments with a rating value greater than or equal to 4.


### Display First Document

In [14]:
# This query returns all the documents where the establishment
# is in London and the 'RatingValue' field has a value greater 
# than or equal to 4.
queryResultsDictionaryList \
    = currentPyMongoCollectionObject \
        .find \
            (queryDictionary)


# This line of code displays the first document from the query 
# results.
pprint \
    (queryResultsDictionaryList[0])

{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650a6b8c5bc6d3e250bfc68a'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/621707',
            'rel': 'self'}]

### High Rating Value, London DataFrame

In [15]:
highRatingLondonDataFrame \
    = pd \
        .DataFrame \
            (list \
                 (queryResultsDictionaryList))


log_function \
    .DebugReturnObjectWriteObject \
        (highRatingLondonDataFrame)

### Number of Rows in High Rating Value, London DataFrame

In [16]:
log_subroutine \
    .PrintAndLogWriteText \
        ('The number of rows in the High Rating Value, London DataFrame is {:,}.' \
            .format \
                (len \
                     (highRatingLondonDataFrame)))

The number of rows in the High Rating Value, London DataFrame is 33.


### Display High Rating Value, London DataFrame

In [17]:
function \
    .ReturnStylerObjectStandardFormat \
        (highRatingLondonDataFrame.head(10),
         'Table 2.2: High Rating Value, London Establishments',
         hideFlagBooleanParameter = True) \
            .hide \
                ('meta', axis = 1)

_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,links
650a6b8c5bc6d3e250bfc68a,"621,707",0,PI/000025307,Charlie's,Other catering premises,"7,841",Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,,ME12,,4,fhrs_4_en-gb,2021-10-18T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceInManagement': 5}",FHRS,"{'longitude': 0.508551, 'latitude': 51.369321}",,"4,627.44",False,"[{'rel': 'self', 'href': 'https://api.ratings.food.gov.uk/establishments/621707'}]"
650a6b8c5bc6d3e250bfc9b0,"1,130,836",0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,"7,841",Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,,TN4 8HR,,5,fhrs_5_en-gb,2021-05-14T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': 0.249255999922752, 'latitude': 51.1236228942871}",,"4,627.87",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/1130836'}]"
650a6b8c5bc6d3e250bfd4fd,"293,783",0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,"7,841",Ferry Road,South Benfleet Essex,,,SS7 1NF,,4,fhrs_4_en-gb,2018-11-02T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 10}",FHRS,"{'longitude': 0.561954, 'latitude': 51.543831}",,"4,631.97",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/293783'}]"
650a6b8c5bc6d3e250bfe2fd,"1,315,095",0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,,RM18 7EH,,5,fhrs_5_en-gb,2020-12-04T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': 0.346336990594864, 'latitude': 51.4640655517578}",,"4,636.85",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/1315095'}]"
650a6b8c5bc6d3e250bfe2fe,"294,474",0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,,RM18 7EH,,5,fhrs_5_en-gb,2018-11-02T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': 0.346325, 'latitude': 51.464078}",,"4,636.85",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/294474'}]"
650a6b8c5bc6d3e250bfea06,"294,900",0,PI/000019066,Mv Valulla,Other catering premises,"7,841",Reeds River Cruises LtdKings ReachRiver ThamesSouthwark,London,,,RM15 5QY,,5,fhrs_5_en-gb,2016-08-23T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': 0.287555, 'latitude': 51.504071}",,"4,640.46",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/294900'}]"
650a6b8c5bc6d3e250c00b14,"293,756",0,PI/000002538,Tereza Joanne,Other catering premises,"7,841",Funcraft UK Ltd King George V Dock Woolwich Manor Way,London,,,E16 2NJ,,5,fhrs_5_en-gb,2021-07-09T00:00:00,508,City of London Corporation,http://www.cityoflondon.gov.uk/Corporation/homepage.htm,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInMa

## 3. What are the top 5 establishments with a rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

### Penang Flavours Latitude and Longitude

In [18]:
# This line of code retrieves the Penang Flavours's document fields, 
# 'latitude' and 'longitude'.
penangFlavoursCoordinatesDictionary \
    = currentPyMongoCollectionObject \
        .find_one \
            ({'BusinessName':'Penang Flavours'}, 
             ['geocode.longitude', 
              'geocode.latitude'])


# This line of code extracts the Penang Flavours's 'latitude' field value.
latitudeFloat \
    = penangFlavoursCoordinatesDictionary \
        ['geocode'] \
        ['latitude']


# This line of code extracts the Penang Flavours's 'longitude' field value.
longitudeFloat \
    = penangFlavoursCoordinatesDictionary \
        ['geocode'] \
        ['longitude']

### Query Search Range

In [19]:
# This line of code establishes the search range as ±0.01 degrees.
degreeSearchFloat = 0.01


# These lines of code calculate and assign the minimum and maximum latitude.
minimumLatitudeFloat \
    = latitudeFloat - degreeSearchFloat

maximumLatitudeFloat \
    = latitudeFloat + degreeSearchFloat


# These lines of code calculate and assign the minimum and maximum longitude.
minimumLongitudeFloat \
    = longitudeFloat - degreeSearchFloat

maximumLongitudeFloat \
    = longitudeFloat + degreeSearchFloat

### Query Results

In [20]:
# These Dictionaries are the query parameters.
queryDictionary \
    = {'RatingValue': 5,
       'geocode.latitude': \
           {'$gte': minimumLatitudeFloat, '$lte': maximumLatitudeFloat},
       'geocode.longitude': \
           {'$gte': minimumLongitudeFloat, '$lte': maximumLongitudeFloat}}
       
sortValuesTupleList \
    = [('scores.Hygiene', 1)]
       
limitInteger \
    = 5


# This query returns the top 5 establishments with a 'RatingValue' field
# value of 5, sorted by lowest hygiene score, nearest to the restaurant,
# 'Penang Flavours'.
resultsPyMongoCursorList \
    = list \
        (currentPyMongoCollectionObject \
            .find \
                (queryDictionary) \
            .sort \
                 (sortValuesTupleList) \
            .limit \
                 (limitInteger))


log_function \
    .DebugReturnObjectWriteObject \
        (resultsPyMongoCursorList)

### Number of Establishments from `count_documents`

In [21]:
numberOfEstablishmentsInteger \
    = currentPyMongoCollectionObject \
        .count_documents \
            (queryDictionary, 
             limit \
                 = limitInteger)

log_subroutine \
    .PrintAndLogWriteText \
        ('Due to the query parameter limiting the number of results, ' \
         + 'there are {:,} establishments with a rating value of 5.' \
            .format \
                (numberOfEstablishmentsInteger))

Due to the query parameter limiting the number of results, there are 5 establishments with a rating value of 5.


### Display All Documents

In [22]:
pprint \
    (resultsPyMongoCursorList)

[{'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'BusinessType': 'Mobile caterer',
  'BusinessTypeID': 7846,
  'ChangesByServerID': 0,
  'Distance': 4646.95593107927,
  'FHRSID': 1380578,
  'LocalAuthorityBusinessID': '14425',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 1SE',
  'RatingDate': '2021-11-11T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('650a6b8c5bc6d3e250c004b1'),
  'geocode': {'latitude': 51.4875335693359, 'longitude': 0.0925370007753372},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1380578',
   

### Highest Rating, Best Hygiene Establishments DataFrame

In [23]:
highRatingBestHygieneDataFrame \
    = pd \
        .DataFrame \
            (resultsPyMongoCursorList)


log_function \
    .DebugReturnObjectWriteObject \
        (highRatingBestHygieneDataFrame)

### Number of Rows in Highest Rating, Best Hygiene Establishments DataFrame

In [24]:
log_subroutine \
    .PrintAndLogWriteText \
        ('The number of rows in the Highest Rating, Best Hygiene DataFrame is {:,}.' \
            .format \
                (len \
                     (highRatingBestHygieneDataFrame)))

The number of rows in the Highest Rating, Best Hygiene DataFrame is 5.


### Display Highest Rating, Best Hygiene Establishments DataFrame

In [25]:
function \
    .ReturnStylerObjectStandardFormat \
        (highRatingBestHygieneDataFrame,
         'Table 2.3: Highest Rating, Best Hygiene Establishments',
         hideFlagBooleanParameter = True) \
            .hide \
                ('meta', axis = 1)

_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,links
650a6b8c5bc6d3e250c004b1,"1,380,578",0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,"7,846",Restaurant And Premises 107A Plumstead High Street,,Plumstead,Greenwich,SE18 1SE,,5,fhrs_5_en-gb,2021-11-11T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': 0.0925370007753372, 'latitude': 51.4875335693359}",,"4,646.96",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/1380578'}]"
650a6b8c5bc6d3e250c004e9,"694,478",0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,"7,844",35 Lakedale Road,,Plumstead,Greenwich,SE18 1PR,,5,fhrs_5_en-gb,2021-06-16T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 5}",FHRS,"{'longitude': 0.0912164, 'latitude': 51.4867296}",,"4,646.97",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/694478'}]"
650a6b8c5bc6d3e250c004e6,"695,241",0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,Greenwich,SE18 1QG,,5,fhrs_5_en-gb,2021-06-16T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 5}",FHRS,"{'longitude': 0.0859939977526665, 'latitude': 51.481517791748}",,"4,646.97",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/695241'}]"
650a6b8c5bc6d3e250c004a1,"695,223",0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,"7,840",144 - 146 Plumstead High Street,,Plumstead,Greenwich,SE18 1JQ,,5,fhrs_5_en-gb,2019-11-13T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5}",FHRS,"{'longitude': 0.0924199968576431, 'latitude': 51.4871482849121}",,"4,646.95",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/695223'}]"
650a6b8c5bc6d3e250c004ce,"694,609",0,PI/000116619,Volunteer,Pub/bar/nightclub,"7,843",130 - 132 Plumstead High Street,,Plumstead,Greenwich,SE18 1JQ,,5,fhrs_5_en-gb,2019-08-05T00:00:00,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}",FHRS,"{'longitude': 0.09208, 'latitude': 51.4873437}",,"4,646.97",False,"[{'rel': 'self', 'href': 'http://api.ratings.food.gov.uk/establishments/694609'}]"


## 4. How many establishments in each Local Authority area have a hygiene score of 0?

### Query Parameters

In [26]:
# 1. This Dictionary matches establishments with a hygiene score of 0.
matchQueryDictionary \
    = {'$match': {'scores.Hygiene': 0}}

# 2. This Dictionary groups the matches by Local Authority and counts 
# how many establishments meet the match criteria for each Local Authority.
groupQueryDictionary \
    = {'$group': \
           {'_id': '$LocalAuthorityName', 
            'count': {'$sum': 1}}}

# 3. This Dictionary sorts the matches in descending order 
# (from highest to lowest) based on count.
sortValuesDictionary \
    = {'$sort': {'count': -1}}

pipeLineList \
    = [matchQueryDictionary, 
       groupQueryDictionary, 
       sortValuesDictionary]

### Query Results

In [27]:
resultsPyMongoCommandCursorList \
    = list \
        (currentPyMongoCollectionObject \
             .aggregate \
                 (pipeLineList))


log_function \
    .DebugReturnObjectWriteObject \
        (resultsPyMongoCommandCursorList)

### Number of Authorities and Establishments from Aggregation Query Results

In [28]:
numberOfAuthoritiesInteger \
    = len \
        ([element for element in resultsPyMongoCommandCursorList \
          if isinstance(element, dict)])

numberOfEstablishmentsInteger \
    = sum \
        ([element['count'] for element in resultsPyMongoCommandCursorList \
          if isinstance(element, dict)])

log_subroutine \
    .PrintAndLogWriteText \
        ('From the aggregation query results, there are {:,} ' \
             .format \
                 (numberOfEstablishmentsInteger)
         + 'establishments with a hygiene score of 0 ' \
         + 'in {:,} Local Authorities.' \
                .format \
                    (numberOfAuthoritiesInteger))

From the aggregation query results, there are 16,827 establishments with a hygiene score of 0 in 55 Local Authorities.


### Number of Authorities and Establishments from `count_documents`

In [29]:
numberOfAuthoritiesInteger \
    = len(currentPyMongoCollectionObject \
            .distinct 
                ('LocalAuthorityName',
                 {'scores.Hygiene': 0}))

numberOfEstablishmentsInteger \
    = currentPyMongoCollectionObject \
        .count_documents \
            ({'scores.Hygiene': 0})

log_subroutine \
    .PrintAndLogWriteText \
        ("From the 'count_documents' method, there are {:,} " \
             .format \
                 (numberOfEstablishmentsInteger)
         + 'establishments with a hygiene score of 0 ' \
         + 'in {:,} Local Authorities.' \
                .format \
                    (numberOfAuthoritiesInteger))

From the 'count_documents' method, there are 16,827 establishments with a hygiene score of 0 in 55 Local Authorities.


### Display Documents

In [30]:
pprint \
    (resultsPyMongoCommandCursorList[0:10])

[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


### Highest Hygiene DataFrame

In [31]:
highestHygieneScoreDataFrame \
    = pd \
        .DataFrame \
            (resultsPyMongoCommandCursorList)


log_function \
    .DebugReturnObjectWriteObject \
        (highestHygieneScoreDataFrame)

In [32]:
log_subroutine \
    .PrintAndLogWriteText \
        ('The number of rows in the Highest Hygiene DataFrame is {:,}.' \
            .format \
                (len \
                     (highestHygieneScoreDataFrame)))

The number of rows in the Highest Hygiene DataFrame is 55.


In [33]:
function \
    .ReturnStylerObjectStandardFormat \
        (highestHygieneScoreDataFrame.head(10),
         'Table 2.4: Local Authorities with Number of Establishments ' \
         + 'with Highest Hygiene Score',
         hideFlagBooleanParameter = True) \
            .hide()

_id,count
Thanet,"1,130"
Greenwich,882
Maidstone,713
Newham,711
Swale,686
Chelmsford,680
Medway,672
Bexley,607
Southend-On-Sea,586
Tendring,542


In [34]:
#log_subroutine \
#    .EndProgramExecution()